# Sourcing and Transforming Data

### Connect to workspace

In [8]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file

ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Ready to use Azure ML 1.13.0 to work with customer_360_ws


### Work with a Datastore

In Azure ML, *datastores* are references to storage locations, such as Azure Storage blob containers. Every workspace has a default datastore - usually the Azure storage blob container that was created with the workspace. If you need to work with data that is stored in different locations, you can add custom datastores to your workspace and set any of them to be the default.

### View Datastores

Run the following code to determine the datastores in your workspace:

In [9]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

ecodatawarehouse - Default = False
azureml_globaldatasets - Default = False
ecodlkstoragebloblive - Default = False
commandjson - Default = False
uploadedresources - Default = False
workspacefilestore - Default = False
workspaceblobstore - Default = True


You can also view and manage datastores in your workspace on the Datastores page for your workspace in [Azure ML Studio](https://ml.azure.com).

### Upload Data to a Datastore

Now that you have determined the available datastores, you can upload files from your local file system to a datastore so that it will be accessible to experiments running in the workspace, regardless of where the experiment script is actually being run.

In [11]:
default_ds.upload_files(files=['./data/flight_delays_data.csv'], # Upload the diabetes csv files in /data
                       target_path='data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Uploading an estimated of 1 files
Uploading ./data/flight_delays_data.csv
Uploaded ./data/flight_delays_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_5284bed764f34f41b61c57689ab01aa4

## Work with Datasets

While you can read data directly from datastores, Azure Machine Learning provides a further abstraction for data in the form of *datasets*. A dataset is a versioned reference to a specific set of data that you may want to use in an experiment. Datasets can be *tabular* or *file*-based.

### Create and Register Tabular Dataset

Let's create a dataset from the flight delays data you uploaded to the datastore. In this case, the data is in a structured format in a CSV file, so we'll use a *tabular* dataset.


Once we create the datasets that reference the flight delays data, you can register it to make it easily accessible to any experiment being run in the workspace.

We'll register the tabular dataset as **flight_delays_data**

In [3]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'flight_delays_data' not in ws.datasets:
    #Create a tabular dataset from the path on the datastore (this may take a short while)
    csv_path = [(default_ds, 'data/flight_delays_data.csv.csv')]
    tab_data_set = Dataset.Tabular.from_delimited_files(path=csv_path)

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='flight_delays_data',
                                description='flight delays data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


Get the flight_delays_data and display first 20 rows examing the content of the data

In [4]:
# Get the training dataset
dataset = ws.datasets.get('flight_delays_data')
dataset = dataset.to_pandas_dataframe()
dataset.head(20)

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,4,19,5,DL,11433,13303,837,-3.0,0.0,1138,1.0,0.0,0.0
1,2013,4,19,5,DL,14869,12478,1705,0.0,0.0,2336,-8.0,0.0,0.0
2,2013,4,19,5,DL,14057,14869,600,-4.0,0.0,851,-15.0,0.0,0.0
3,2013,4,19,5,DL,15016,11433,1630,28.0,1.0,1903,24.0,1.0,0.0
4,2013,4,19,5,DL,11193,12892,1615,-6.0,0.0,1805,-11.0,0.0,0.0
5,2013,4,19,5,DL,10397,15016,1726,-1.0,0.0,1818,-19.0,0.0,0.0
6,2013,4,19,5,DL,15016,10397,1900,0.0,0.0,2133,-1.0,0.0,0.0
7,2013,4,19,5,DL,10397,14869,2145,15.0,1.0,2356,24.0,1.0,0.0
8,2013,4,19,5,DL,10397,10423,2157,33.0,1.0,2333,34.0,1.0,0.0
9,2013,4,19,5,DL,11278,10397,1900,323.0,1.0,2055,322.0,1.0,0.0


let's do a quick description of the features available.

In [5]:
dataset.describe()

,Year,Month,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
count,2719418.0,2.719418e+06,2.719418e+06,2.719418e+06,2.719418e+06,2.719418e+06,2.719418e+06,2.691974e+06,2.691974e+06,2.719418e+06,2.690385e+06,2.719418e+06,2.719418e+06
mean,2013.0,6.979968e+00,1.579747e+01,3.898391e+00,1.274226e+04,1.274246e+04,1.326645e+03,1.053687e+01,2.023419e-01,1.505270e+03,6.637688e+00,2.166316e-01,1.067618e-02
std,0.0,1.984331e+00,8.799860e+00,1.985988e+00,1.501973e+03,1.501969e+03,4.713766e+02,3.609953e+01,4.017458e-01,4.939662e+02,3.864881e+01,4.119496e-01,1.027726e-01
min,2013.0,4.000000e+00,1.000000e+00,1.000000e+00,1.014000e+04,1.014000e+04,1.000000e+00,-6.300000e+01,0.000000e+00,1.000000e+00,-9.400000e+01,0.000000e+00,0.000000e+00
25%,2013.0,5.000000e+00,8.000000e+00,2.000000e+00,1.129200e+04,1.129200e+04,9.200000e+02,-4.000000e+00,0.000000e+00,1.120000e+03,-1.100000e+01,0.000000e+00,0.000000e+00
50%,2013.0,7.000000e+00,1.600000e+01,4.000000e+00,1.289200e+04,1.289200e+04,1.320000e+03,-1.000000e+00,0.000000e+00,1.528000e+03,-3.000000e+00,0.000000e+00,0.000000e+00
75%,2013.0,9.000000e+00,2.300000e+01,6.000000e+00,1.405700e+04,1.405700e+04,1.725000e+03,9.000000e+00,0.000000e+00,1.918000e+03,1.000000e+01,0.000000e+00,0.000000e+00
max,2013.0,1.000000e+01,3.100000e+01,7.000000e+00,1.537600e+04,1.537600e+04,2.359000e+03,1.863000e+03,1.000000e+00,2.359000e+03,1.845000e+03,1.000000e+00,1.000000e+00


Displaying a information of the dataset will help us know which columns need to be engineered.

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2719418 entries, 0 to 2719417
Data columns (total 14 columns):
Year               int64
Month              int64
DayofMonth         int64
DayOfWeek          int64
Carrier            object
OriginAirportID    int64
DestAirportID      int64
CRSDepTime         int64
DepDelay           float64
DepDel15           float64
CRSArrTime         int64
ArrDelay           float64
ArrDel15           float64
Cancelled          float64
dtypes: float64(5), int64(8), object(1)
memory usage: 290.5+ MB


### Feature Engineering

Feature engineering here will include removing target leakers and features that are not useful to our hypothesis. 
We will then make sure the columns(features) are of the right data types for the algorithm to be used for the prediction.

In [7]:
# Get the training dataset
dataset = ws.datasets.get('flight_delays_data')
dataset = dataset.to_pandas_dataframe().dropna()

# Remove target leaker and features that are not useful
target_leakers = ['DepDel15','ArrDelay','Cancelled','Year']
dataset.drop(columns=target_leakers, axis=1, inplace=True)

# convert some columns to categorical features
columns_as_categorical = ['OriginAirportID','DestAirportID','ArrDel15']
dataset[columns_as_categorical] = dataset[columns_as_categorical].astype('object')

# The labelEncoder and OneHotEncoder only works on categorical features. We need first to extract the categorial featuers using boolean mask.
categorical_feature_mask = dataset.dtypes == object 
categorical_cols = dataset.columns[categorical_feature_mask].tolist()
categorical_cols

['Carrier', 'OriginAirportID', 'DestAirportID', 'ArrDel15']

LabelEncoder converts each class under specified feature to a numerical value. 

Let’s go through the steps to see how to do it.

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Apply LabelEncoder on each of the categorical columns:
dataset[categorical_cols] = dataset[categorical_cols].apply(lambda col:le.fit_transform(col))
dataset[categorical_cols].head(10)

,Carrier,OriginAirportID,DestAirportID,ArrDel15
0,4,18,36,0
1,4,64,27,0
2,4,48,64,0
3,4,67,18,1
4,4,13,29,0
5,4,2,67,0
6,4,67,2,0
7,4,2,64,1
8,4,2,3,1
9,4,15,2,1


In [9]:
# Drop all null values
dataset.dropna(inplace=True)
dataset.head(20)

,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,CRSArrTime,ArrDel15
0,4,19,5,4,18,36,837,-3.0,1138,0
1,4,19,5,4,64,27,1705,0.0,2336,0
2,4,19,5,4,48,64,600,-4.0,851,0
3,4,19,5,4,67,18,1630,28.0,1903,1
4,4,19,5,4,13,29,1615,-6.0,1805,0
5,4,19,5,4,2,67,1726,-1.0,1818,0
6,4,19,5,4,67,2,1900,0.0,2133,0
7,4,19,5,4,2,64,2145,15.0,2356,1
8,4,19,5,4,2,3,2157,33.0,2333,1
9,4,19,5,4,15,2,1900,323.0,2055,1


Doing a relative data split based on the Month column

In [12]:
train_ds, test_ds = dataset.loc[dataset['Month'] < 9], dataset.loc[dataset['Month'] >= 9]
train_count = train_ds.Month.count()
test_count = test_ds.Month.count()
print('Test data ratio:',(test_count/(test_count+train_count))*100)

Test data ratio: 27.553194059586268
